In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import clone
from sklearn.linear_model import Lasso, Ridge
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error

### Data preparation

In [28]:
data_path = '../data/real_state_data_after_cleaning.csv'
df = pd.read_csv(data_path)
print(df.columns)
print(df.shape)

Index(['precio', 'ambientes', 'no_baños', 'terreno_m2', 'año_constr',
       'no_dormitorios', 'area_constr_m2', 'estacionamientos', 'latitud',
       'longitud', 'tipo_de_propiedad', 'ofertado_como', 'ciudad', 'zona',
       'provincia', 'dpto'],
      dtype='object')
(1522, 16)


Se excluyen las variables con una sola categoría

In [29]:
# Variables a considerar en el modelo
numerical = ['precio', 'ambientes', 'no_baños', 'terreno_m2', 'año_constr',
       'no_dormitorios', 'area_constr_m2', 'estacionamientos', 'latitud',
       'longitud']
num_continuous = ['precio', 'terreno_m2', 'area_constr_m2',  'latitud', 'longitud']
num_discrete = ['ambientes', 'no_baños', 'año_constr',
       'no_dormitorios', 'estacionamientos']
categorical = ['tipo_de_propiedad', 'ciudad', 'zona']


In [30]:
#df_1 solo las variables consideradas
df_1 = df[numerical + categorical]


Se eliminan las filas con valores faltantes, con excepción de dos columnas

In [31]:
#eliminando filas nulas en todas las columnas que nos sean ambientes y estacionamiento
#df_1.isna().sum()
df_2 = df_1.dropna(subset=df_1.columns.difference(['ambientes', 'terreno_m2', 'estacionamientos'])).copy()
df_2.isna().sum()

precio                  0
ambientes             795
no_baños                0
terreno_m2            758
año_constr              0
no_dormitorios          0
area_constr_m2          0
estacionamientos     1159
latitud                 0
longitud                0
tipo_de_propiedad       0
ciudad                  0
zona                    0
dtype: int64

In [32]:
#remplazando los valores faltantes de terreno por los de área de construcción
df_2['terreno_m2'].fillna(df['area_constr_m2'], inplace=True)
#remplazando los valores faltantes de estacionamiento por 0
df_2['estacionamientos'].fillna(0, inplace=True)
df_2.isna().sum()


precio                 0
ambientes            795
no_baños               0
terreno_m2             0
año_constr             0
no_dormitorios         0
area_constr_m2         0
estacionamientos       0
latitud                0
longitud               0
tipo_de_propiedad      0
ciudad                 0
zona                   0
dtype: int64

Para ambientes se usará knn-imputer, se ve que hay entradas donde el número de ambientes sí es menor que la suma de dormitorios y baños. n para KNN es un hiperparámetro y se determinará cuando se entrené el modelo.

In [33]:
df_2[df_2['ambientes']<(df_2['no_baños']+df_2['no_dormitorios'])].shape

(93, 13)

En la variables zona y tipo_de_propiedad, se eliminan las categorias con menos de 5 elementos

In [34]:
s_zonas = df_2.zona.value_counts()
zonas = [zona for zona, value in s_zonas.items() if value > 4]
df_3 = df_2[df_2['zona'].isin(zonas)]
tipo_de_propiedades = df_3.tipo_de_propiedad.value_counts()
propiedades = [propiedad for propiedad, value in tipo_de_propiedades.items() if value > 4]
df_3 = df_3[df_3['tipo_de_propiedad'].isin(propiedades)]
df_3.tipo_de_propiedad.value_counts()#.to_frame().T

tipo_de_propiedad
Departamento                  749
Casa                          658
Casa con Espacio Comercial     54
Estudio/Monoambiente            9
Name: count, dtype: int64

Convirtiendo los datos al formato deseado

In [35]:
df_4 = df_3.copy()
df_4['terreno_m2'] = pd.to_numeric(df_3['terreno_m2'].str.replace(',', '.'), errors='raise')
df_4['area_constr_m2'] = pd.to_numeric(df_3['area_constr_m2'].str.replace(',', '.'), errors='coerce')
df_4[num_discrete] = df_3[num_discrete].apply(pd.to_numeric, errors='coerce').astype('Int64')
print(df_4.dtypes)

precio               float64
ambientes              Int64
no_baños               Int64
terreno_m2           float64
año_constr             Int64
no_dormitorios         Int64
area_constr_m2       float64
estacionamientos       Int64
latitud              float64
longitud             float64
tipo_de_propiedad     object
ciudad                object
zona                  object
dtype: object


También, se eliminan los precios por debajo de 20000$, y se filtran los valores de longitud y latitud muy diferentes a los comunes

In [36]:
df_5 = df_4[df_4['precio']>20000].copy()
print(df_5.shape)
df_5 = df_5[(df_5['longitud']>-64) & (df_5['longitud']<-63)].copy()
df_5 = df_5[(df_5['latitud']>-18) & (df_5['latitud']<-17)].copy()
print(df_5.shape)

(1456, 13)
(1445, 13)


Como último paso, se para los datos faltantes en ambientes la suma de baños y dormitorios.



In [88]:
df_6 = df_5.copy()

mask_nan = df_6.ambientes.isna()
mask_suma = df_6.ambientes==(df_6.no_baños+df_6.no_dormitorios)

print('Se cumple la igualdad antes', df_6[mask_suma].shape)
print('Datos faltantes antes', df_6.ambientes[mask].shape)

df_6.ambientes[mask_nan] = df_6.no_baños[mask_nan] + df_6.no_dormitorios[mask_nan]

print('Se cumple la igualdad despues', df_6[mask_suma].shape)

#df_6[['ambientes', 'no_baños', 'no_dormitorios']][mask_nan]

Se cumple la igualdad antes (134, 13)
Datos faltantes antes (773,)
Se cumple la igualdad despues (134, 13)


In [91]:
# final check for missing data
df_6.isna().sum().sum()

0

df_6 se usará para la construcción del modelo

In [92]:
df_6.to_csv('../data/real_state_data_for_model.csv',index=False)

### Construccion del modelo

Se probarán varios algoritmos de regresión y se usará r como la métrica de evaluación.

In [39]:
#reproducibility
random_seed = 42  # Set a random seed for reproducibility
np.random.seed(random_seed)

#separando la variable objetivo
X = df_6.drop('precio', axis=1)
y = df_6['precio']
y_stratify = df_6[['tipo_de_propiedad']] #mantener las proporciones de tipo_de_propiedad

#preprocesando los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, 
                                                    random_state=random_seed,
                                                     stratify=y_stratify)

#variables para las transformaciones, 
numerical = ['no_baños', 'terreno_m2', 'año_constr', 
       'no_dormitorios', 'area_constr_m2', 'latitud', 'longitud']
categorical = ['tipo_de_propiedad', 'ciudad', 'zona']

#generando el preprocesador por columna
numeric_transformer = Pipeline([('inputer', SimpleImputer(strategy='constant', fill_value=0)),
                                ('scaler', StandardScaler())])
cat_transformer = Pipeline([('inputer', SimpleImputer(strategy='most_frequent')),
                            ('onehot', OneHotEncoder())])#handle_unknown='ignore'
estcmnts_transformer = Pipeline([('inputer', SimpleImputer(strategy='constant', fill_value=0)),
                               ('scaler', StandardScaler())])
ambientes_transformer = Pipeline([('imputer', KNNImputer(n_neighbors=5)),
                                  ('scaler', StandardScaler())])

preprocessor = ColumnTransformer([('num', numeric_transformer, numerical),
                                ('cat', cat_transformer, categorical),
                                ('est', estcmnts_transformer, ['estacionamientos']),
                                ('amb', ambientes_transformer, ['ambientes'])
                                ])
models = {
    #'lasso': Lasso(max_iter=100000, random_state=random_seed),
    #'ridge': Ridge(random_state=random_seed),
    #'random_forest': RandomForestRegressor(random_state=random_seed),
    #'svr' : SVR(),
    'gradboost' : GradientBoostingRegressor(random_state=random_seed),
    #'knn' : KNeighborsRegressor()
}

param_grid = {
    'lasso': {'model__alpha': [0.1, 1.0, 10.0], 
              'preprocessor__amb__imputer__n_neighbors': [3, 5, 7]},
    'ridge': {'model__alpha': [0.1, 1.0, 10.0],
              'preprocessor__amb__imputer__n_neighbors': [3, 5, 7]},
    'random_forest': {'model__n_estimators': [100, 200, 300, 400, 500],
                      'model__max_depth' : [1,2,3,4,5,6,None],
                      'preprocessor__amb__imputer__n_neighbors': [3, 5, 7]},
    'svr' : {'model__C': [0.1, 1.0, 10.0],  
            'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  
            'model__gamma': ['scale', 'auto'], 
            'model__epsilon': [0.1, 0.2, 0.5], 
            'preprocessor__amb__imputer__n_neighbors': [3, 5, 7]},
    'gradboost' : {'model__learning_rate': [0.1, 0.8], 
            'model__n_estimators': [500, 600, 700],
            'model__max_depth': [2, 3, 4, 5],  
            #'model__subsample': [0.8, 1.0], 
            #'model__loss': ['squared_error', 'absolute_error'],
            'preprocessor__amb__imputer__n_neighbors': [3]},
    'knn' : {'model__n_neighbors' : [2, 3, 4, 5, 6, 7, 8, 9, 10],
             'model__weights' : ['uniform', 'distance'],
             'model__p' : [1, 2, 3, 4, 5],
             'preprocessor__amb__imputer__n_neighbors': [3, 5, 7]}
}

results = {}
for model_name, model in models.items():
    pipeline = Pipeline([('preprocessor', preprocessor), ('model', model)])
    grid_search = GridSearchCV(pipeline, param_grid[model_name], cv=5, n_jobs=-1)

    grid_search.fit(X_train, y_train)

    results[model_name] = {
        'best_model' : grid_search.best_estimator_,
        'best_params': grid_search.best_params_,
        'best_score': grid_search.best_score_,
        'test_score': grid_search.score(X_test, y_test),
        'MAE': mean_absolute_error(y_test, grid_search.best_estimator_.predict(X_test))
    }

for model_name, result in results.items():
    print(f"Model: {model_name}")
    print(f"Best Parameters: {result['best_params']}")
    print(f"Best Score: {result['best_score']}")
    print(f"Test Score: {result['test_score']}")
    print(f"MAE: {result['MAE']}")
    print()

Model: gradboost
Best Parameters: {'model__learning_rate': 0.1, 'model__max_depth': 5, 'model__n_estimators': 600, 'preprocessor__amb__imputer__n_neighbors': 3}
Best Score: 0.8705536649686823
Test Score: 0.8967057931417916
MAE: 4203.376353877931



Model: random_forest
Best Parameters: {'model__max_depth': None, 'model__n_estimators': 300, 'preprocessor__amb__imputer__n_neighbors': 3}
Best Score: 0.8688338842652742
Test Score: 0.8705882428055841
MAE: 5000.085156175565

Model: gradboost
Best Parameters: {'model__learning_rate': 0.1, 'model__max_depth': 5, 'model__n_estimators': 600, 'preprocessor__amb__imputer__n_neighbors': 3}
Best Score: 0.8705536649686823
Test Score: 0.8967057931417916
MAE: 4203.376353877931

Model: knn
Best Parameters: {'model__n_neighbors': 7, 'model__p': 1, 'model__weights': 'distance', 'preprocessor__amb__imputer__n_neighbors': 3}
Best Score: 0.85815282433633
Test Score: 0.8533985552044425
MAE: 4969.396716026311


El mejor modelo es Gradient Boosting Regressor, por lo que se toma este.

In [40]:
best_model = results['gradboost']['best_model']
best_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('inputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['no_baños', 'terreno_m2',
                                                   'año_constr',
                                                   'no_dormitorios',
                                                   'area_constr_m2', 'latitud',
                                                   'longitud']),
                                                 ('cat',
                                                  Pipeline(steps=[('inputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('o...
                                                  ['tipo_de_propiedad',
                                                   'ciudad', 'zona']),
                                                 ('est',
                                                  Pipeline(steps=[('inputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['estacionamientos']),
                                                 ('amb',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=3)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['ambientes'])])),
                ('model',
                 GradientBoostingRegressor(max_depth=5, n_estimators=600,
                                           random_state=42))])

Prediciendo con el mejor modelo

In [41]:
data = {
    'ambientes': [7],
    'no_baños': [2],
    'terreno_m2': [393],
    'año_constr': [2013],
    'no_dormitorios': [3],
    'area_constr_m2': [100.0],
    'estacionamientos': [4],
    'latitud': [-17.643187],
    'longitud': [-63.173777],
    'tipo_de_propiedad': ['Casa'],
    'ciudad': ['Santa Cruz de la Sierra'],
    'zona': ['Norte']
}
#imput_data: dataframe
data_1 = pd.DataFrame(data)

best_model.predict(data_1)

array([60069.00203555])

In [42]:
data = {
    'ambientes': [5],
    'no_baños': [2],
    'terreno_m2': [60],
    'año_constr': [2018],
    'no_dormitorios': [2],
    'area_constr_m2': [60],
    'estacionamientos': [1],
    'latitud': [-17.787357],
    'longitud': [-63.213448],
    'tipo_de_propiedad': ['Departamento'],
    'ciudad': ['Santa Cruz de la Sierra'],
    'zona': ['Oeste']
}
data_2 = pd.DataFrame(data)

best_model.predict(data_2)


array([69045.51427282])

Exportando el modelo

In [43]:
import pickle
import numpy as np
pickl = {'model': best_model}
file_name = '../models/model_file_test.p'
pickle.dump( pickl, open(file_name, "wb" ) )

Probando el modelo exportado

In [44]:
with open(file_name, 'rb') as pickled:
    data = pickle.load(pickled)
    model = data['model']

model.predict(data_1)


array([60069.00203555])

### Debugging

In [45]:
for i in df_6.zona.value_counts().index:
    print(i)

Norte
Sur
Este
Equipetrol/NorOeste
Urubo
Oeste
Sureste
Urbari
Centro (Casco Viejo)
ESTE
Noreste
Suroeste
Noroeste


In [46]:
#inspeccionando las columnas
scaler = preprocessor.fit(X_train)
X_train_trans = scaler.transform(X_train)
X_test_trans = scaler.transform(X_test)
df_transf = pd.DataFrame(X_train_trans)
def get_column_names(column_transformer):
  features_names =[]
  for key, value in column_transformer.named_transformers_.items():
    features = list(value.get_feature_names_out())
    features_names.extend(features)
  return features_names
df_transf.columns = get_column_names(preprocessor)
df_transf[['tipo_de_propiedad_Casa',
       'tipo_de_propiedad_Casa con Espacio Comercial',
       'tipo_de_propiedad_Departamento',
       'tipo_de_propiedad_Estudio/Monoambiente']]
#print(y_stratify.value_counts())
#df_transf[['tipo_de_propiedad_Departamento']].sum() #proporciones se mantienen según tipo de propiedad

,tipo_de_propiedad_Casa,tipo_de_propiedad_Casa con Espacio Comercial,tipo_de_propiedad_Departamento,tipo_de_propiedad_Estudio/Monoambiente
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,0.0
...,...,...,...,...
1078,1.0,0.0,0.0,0.0
1079,1.0,0.0,0.0,0.0
1080,1.0,0.0,0.0,0.0
1081,1.0,0.0,0.0,0.0


In [47]:
import pandas as pd

columns = ['ambientes', 'no_baños', 'terreno_m2', 'año_constr', 'no_dormitorios',
           'area_constr_m2', 'estacionamientos', 'latitud', 'longitud',
           'tipo_de_propiedad', 'ciudad', 'zona']

data = [[5, 2, 60, 2018, 2, 60, 1, -17.787357, -63.213448, 'Departamento',
     'Santa Cruz de la Sierra', 'Oeste']]

data_2 = pd.DataFrame(data, columns=columns)
input = ['12', '12', '12', '12', '12', '12.3', '12', '12.43', '12.3', 'Departamento', 'Santa Cruz de la Sierra', 'Norte']

# Convert numbers to integers or floats
converted_input = []
for x in input:
    if x.isdigit():
        converted_input.append(int(x))
    else:
        try:
            converted_input.append(float(x))
        except ValueError:
            converted_input.append(x)

print(converted_input)


[12, 12, 12, 12, 12, 12.3, 12, 12.43, 12.3, 'Departamento', 'Santa Cruz de la Sierra', 'Norte']


In [48]:
input = ['12', '12', '12', '12', '12',
          '12.3', '12', '12.43', '12.3', 'Departamento', 
          'Santa Cruz de la Sierra', 'Norte']

def predictor(input):
    columns = ['ambientes', 'no_baños', 'terreno_m2', 'año_constr', 'no_dormitorios',
           'area_constr_m2', 'estacionamientos', 'latitud', 'longitud',
           'tipo_de_propiedad', 'ciudad', 'zona']
    converted_input = []
    for x in input:
        if x.isdigit():
            converted_input.append(int(x))
        else:
            try:
                converted_input.append(float(x))
            except ValueError:
                converted_input.append(x)
    data = [converted_input]
    data_df = pd.DataFrame(data, columns=columns)
    print(data_df)
    loaded_model = pickle.load(open("model_file.p","rb"))
    result = loaded_model['model'].predict(data_df)
    return int(result)

predictor(input)

   ambientes  no_baños  terreno_m2  año_constr  no_dormitorios   
0         12        12          12          12              12  \

   area_constr_m2  estacionamientos  latitud  longitud tipo_de_propiedad   
0            12.3                12    12.43      12.3      Departamento  \

                    ciudad   zona  
0  Santa Cruz de la Sierra  Norte  


FileNotFoundError: [Errno 2] No such file or directory: 'model_file.p'

In [ ]:
numerical = ['estacionamientos','ambientes', 'no_baños', 'terreno_m2', 'año_constr',
       'no_dormitorios', 'area_constr_m2', 'latitud', 'longitud']
df_6 = df_6.drop(df_6[df_6['estacionamientos']==90].index)
df_6.shape